There will be a review of some features of pandas that may be helpfull for data wrangling, model fitting and scoring
Also, there will be a short introduction to popular modeling toolkits:
- statsmodels
- scikitlearn

### Interfacing between pandas and Model Code
Common workflow for model development:
- Use pandas for data loading and cleaning. 
  Feature engineering -> Is any data transformation or analytics that extract information from a raw dataset that may be useful in a modeling context. Important part of the model development in ml. Data aggregation used with groupby are often used in a feature engineering context

The way to connect pandas and othe analysis libraries is usually NumPy arrays.
We have to turn a DataFrame into a NumPy array

In [19]:
import numpy as np
import pandas as pd

In [20]:
data = pd.DataFrame({
    "x0":np.arange(5)+1,
    "x1":[0.01,-0.01,0.25,-4.1,0.],
    "y":[-1.5,0.,3.6,1.3,-2.]})
data

,x0,x1,y
0,1,0.01,-1.5
1,2,-0.01,0.0
2,3,0.25,3.6
3,4,-4.10,1.3
4,5,0.00,-2.0


In [21]:
data.columns

Index(['x0', 'x1', 'y'], dtype='object')

In [22]:
#Passing the DataFrame to numpy
np_array = data.to_numpy()
np_array

array([[ 1.  ,  0.01, -1.5 ],
       [ 2.  , -0.01,  0.  ],
       [ 3.  ,  0.25,  3.6 ],
       [ 4.  , -4.1 ,  1.3 ],
       [ 5.  ,  0.  , -2.  ]])

In [23]:
#Going back to DataFrame
df2 = pd.DataFrame(np_array,columns=['x0','x1','y'])
df2


,x0,x1,y
0,1.0,0.01,-1.5
1,2.0,-0.01,0.0
2,3.0,0.25,3.6
3,4.0,-4.10,1.3
4,5.0,0.00,-2.0


to_numpy is intended to be used when your data is homogeneous (ex: all numeric types)
If you have heterogneous data, the result will be an ndarray of Python objects

In [24]:
df3 = data.copy()
df3['strings'] =['a','b','c','d','e']
df3

,x0,x1,y,strings
0,1,0.01,-1.5,a
1,2,-0.01,0.0,b
2,3,0.25,3.6,c
3,4,-4.10,1.3,d
4,5,0.00,-2.0,e


In [25]:
df3.to_numpy()

array([[1, 0.01, -1.5, 'a'],
       [2, -0.01, 0.0, 'b'],
       [3, 0.25, 3.6, 'c'],
       [4, -4.1, 1.3, 'd'],
       [5, 0.0, -2.0, 'e']], dtype=object)

In [26]:
# In some models we may wish to only use some columns
# We can use loc with to_numpy
model_cols=['x0','x1']
data.loc[:,model_cols].to_numpy()

array([[ 1.  ,  0.01],
       [ 2.  , -0.01],
       [ 3.  ,  0.25],
       [ 4.  , -4.1 ],
       [ 5.  ,  0.  ]])

Some libraries have native support with pandas and automatically convert from DataFrame to NumPy array and attach model parameter names to the columns of output tables or series.

In [27]:
# We have a non nonnumeric column 
data['category'] = pd.Categorical(['a','b','a','a','b'],categories=['a','b'])
data

,x0,x1,y,category
0,1,0.01,-1.5,a
1,2,-0.01,0.0,b
2,3,0.25,3.6,a
3,4,-4.10,1.3,a
4,5,0.00,-2.0,b


In [28]:
dummies = pd.get_dummies(data.category,prefix="cat")
data_with_dummies =  data.drop("category",axis=1).join(dummies)
data_with_dummies

,x0,x1,y,cat_a,cat_b
0,1,0.01,-1.5,True,False
1,2,-0.01,0.0,False,True
2,3,0.25,3.6,True,False
3,4,-4.10,1.3,True,False
4,5,0.00,-2.0,False,True


### Creating Model Descriptions with Patsy
Patsy is a Python library for decribing statistical models (especially linear models).
It uses string base formula syntax of R.
Patsy syntax -> y ~ x0 + x1.
patsy.dmatrices functions -> Takes formula string along with dataset

In [29]:
data = pd.DataFrame({
    "x0":np.arange(1,6),
    "x1":[0.01,-0.01,0.25,-4.10,0.00],
    "y":[-1.5,0.0,3.6,1.3,-2.0]
})
data

,x0,x1,y
0,1,0.01,-1.5
1,2,-0.01,0.0
2,3,0.25,3.6
3,4,-4.10,1.3
4,5,0.00,-2.0


In [32]:
import patsy
y, X = patsy.dmatrices("y ~ x0 + x1",data)

In [33]:
y

DesignMatrix with shape (5, 1)
     y
  -1.5
   0.0
   3.6
   1.3
  -2.0
  Terms:
    'y' (column 0)

In [34]:
X

DesignMatrix with shape (5, 3)
  Intercept  x0     x1
          1   1   0.01
          1   2  -0.01
          1   3   0.25
          1   4  -4.10
          1   5   0.00
  Terms:
    'Intercept' (column 0)
    'x0' (column 1)
    'x1' (column 2)

These Patsy DesignMatrix instances are Numpy ndarrays with metadata

In [35]:
np.asarray(y)

array([[-1.5],
       [ 0. ],
       [ 3.6],
       [ 1.3],
       [-2. ]])

In [37]:
np.asarray(X)

array([[ 1.  ,  1.  ,  0.01],
       [ 1.  ,  2.  , -0.01],
       [ 1.  ,  3.  ,  0.25],
       [ 1.  ,  4.  , -4.1 ],
       [ 1.  ,  5.  ,  0.  ]])

In [38]:
#Patsy object can be passed directly into algorithms like numpy.linalg.lstsq -> least squares regression
coef,resid,_,_ = np.linalg.lstsq(X,y)

C:\Users\bruno\AppData\Local\Temp\ipykernel_2944\958575462.py:2: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  coef,resid,_,_ = np.linalg.lstsq(X,y)


In [41]:
coef 

array([[ 0.31290976],
       [-0.07910564],
       [-0.26546384]])

In [43]:
# model metada is retained in design_info attribute. We can reattach the model columns names to the fitted coefficients to obtain a Series
coef = pd.Series(coef.squeeze(),index= X.design_info.column_names)

In [44]:
coef

Intercept    0.312910
x0          -0.079106
x1          -0.265464
dtype: float64

### Data Transformations in Patsy Formulas
We can mix python code into Patsy formulas. 

In [52]:
y,X = patsy.dmatrices('y ~ x0 + np.log(np.abs(x1) + 1)', data) #Patsy interprets de python function

#Commonly used variable transformations -> 
- standarizing (mean 0 and variance 1)
- centering (substracting the mean)

In [53]:
y,X = patsy.dmatrices('y ~ standardize(x0) + center(x1)',data)
data

,x0,x1,y
0,1,0.01,-1.5
1,2,-0.01,0.0
2,3,0.25,3.6
3,4,-4.10,1.3
4,5,0.00,-2.0


In [54]:
X

DesignMatrix with shape (5, 3)
  Intercept  standardize(x0)  center(x1)
          1         -1.41421        0.78
          1         -0.70711        0.76
          1          0.00000        1.02
          1          0.70711       -3.33
          1          1.41421        0.77
  Terms:
    'Intercept' (column 0)
    'standardize(x0)' (column 1)
    'center(x1)' (column 2)

When applying transformations like center and standarize, you should be careful when using this information to form predictions based on new data.
These are called stateful transformations-> Must use statistics like mean or standard deviation of the original dataset when transforming a new dataset.
patsy.build_design_matrices function applies transformations yo new out-of-sample data, using the saved information from the original in-sample dataset

In [55]:
new_data = pd.DataFrame({
    'x0':np.arange(6,10),
    'x1':[3.1,-0.5,0,2.3],
    'y': np.arange(1,5)
})
new_data

,x0,x1,y
0,6,3.1,1
1,7,-0.5,2
2,8,0.0,3
3,9,2.3,4


In [57]:
new_X = patsy.build_design_matrices([X.design_info],new_data)
new_X

[DesignMatrix with shape (4, 3)
   Intercept  standardize(x0)  center(x1)
           1          2.12132        3.87
           1          2.82843        0.27
           1          3.53553        0.77
           1          4.24264        3.07
   Terms:
     'Intercept' (column 0)
     'standardize(x0)' (column 1)
     'center(x1)' (column 2)]

### Categorical Data and Patsy
Nonnuemeric data can be transformed for a model design matrix in different ways.
When using nonnumeric terms in a Patsy formula, the get converted into dummy variables by default

In [59]:
data = pd.DataFrame({
        'key1':['a','a','b','b','a','b','a','b'],
        'key2': [0,1,0,1,0,1,0,0],
        'v1':np.arange(1,9),
        'v2': [-1,0,2.5,-0.5,4.0,-1.2,0.2,-1.7]
    })
data

,key1,key2,v1,v2
0,a,0,1,-1.0
1,a,1,2,0.0
2,b,0,3,2.5
3,b,1,4,-0.5
4,a,0,5,4.0
5,b,1,6,-1.2
6,a,0,7,0.2
7,b,0,8,-1.7


In [61]:
y,X = patsy.dmatrices('v2 ~ key1', data)
X

DesignMatrix with shape (8, 2)
  Intercept  key1[T.b]
          1          0
          1          0
          1          1
          1          1
          1          0
          1          1
          1          0
          1          1
  Terms:
    'Intercept' (column 0)
    'key1' (column 1)

### Introduction to statsmodels
Python library for:
- Fit many kinds of statistical models
- Perform statistical tests
- Data exploration and visualization
It contains mode "classical" frequentist statistical methods of machine learning

### Some models
- Linear models, generalized linear models and robust linear models
- Linear mixed effects models
- Analusysi of variance (ANOVA) methods
- Time series processes and state space models
- Generalized method of moments

### Estimating linear models
Linear models in statsmodels.
Two main interfaces:
- Array based
- Formula based
  

In [63]:
import statsmodels.api as sm
import statsmodels.formula.api as smf

In [64]:
# Generating random data for the linear model
rng = np.random.default_rng(seed=12345)
rng

Generator(PCG64) at 0x2038D9BAB20

In [65]:
def dnorm(mean,variance,size=1):
    if isinstance(size,int):
        size = size,
    return mean + np.sqrt(variance) * rng.standard_normal(*size)

In [87]:
#dnorm -> Helper function for generating normally distibuted data with particular mean and variance
N = 100
X = np.c_[dnorm(0,0.4,size=N),
          dnorm(0,0.6,size=N),
          dnorm(0,0.2,size=N)
          ]
eps = dnorm(0,0.1,size=N)
beta = [0.1,0.3,0.5]
y = np.dot(X,beta) + eps


In [89]:
X[:5]

array([[ 0.57442228,  0.11281981,  0.04436047],
       [-1.20687418,  0.53490757, -0.05271562],
       [ 0.12948879,  0.68119995,  0.83801711],
       [-0.20801298, -0.51638154, -0.16675323],
       [ 0.58914102,  0.51800859,  0.46263005]])

In [90]:
y[:5]

array([ 0.01983959,  0.19390324,  0.65222324, -0.29719418,  0.20177841])

In [92]:
#sm.add_constant function -> adds an intercept column to an existing matrix
X_model = sm.add_constant(X)
X_model

array([[ 1.00000000e+00,  5.74422282e-01,  1.12819806e-01,
         4.43604694e-02],
       [ 1.00000000e+00, -1.20687418e+00,  5.34907574e-01,
        -5.27156181e-02],
       [ 1.00000000e+00,  1.29488789e-01,  6.81199951e-01,
         8.38017107e-01],
       [ 1.00000000e+00, -2.08012982e-01, -5.16381540e-01,
        -1.66753234e-01],
       [ 1.00000000e+00,  5.89141025e-01,  5.18008586e-01,
         4.62630052e-01],
       [ 1.00000000e+00,  1.17990194e-01, -4.82207811e-01,
         1.99868394e-01],
       [ 1.00000000e+00,  1.39913103e+00, -8.94998736e-01,
         9.32921043e-01],
       [ 1.00000000e+00,  1.81483453e+00, -7.53280980e-01,
        -8.67628955e-02],
       [ 1.00000000e+00, -3.68031507e-01, -1.50996680e+00,
         3.36629558e-01],
       [ 1.00000000e+00,  4.10664766e-01,  1.75309163e-02,
         2.73096094e-01],
       [ 1.00000000e+00, -6.18177278e-01, -1.43369321e+00,
        -1.43268135e+00],
       [ 1.00000000e+00, -6.31127485e-01,  1.35793340e-01,
      

In [95]:
# sm.OLS class can fit an ordinary least squares linear regression
model = sm.OLS(y,X)

In [98]:
#Models fit method returns a regression results object containing estimated model parameters
results = model.fit()

In [99]:
results.params

array([0.07734696, 0.31137941, 0.47987929])

In [102]:
print(results.summary())

                                 OLS Regression Results                                
Dep. Variable:                      y   R-squared (uncentered):                   0.597
Model:                            OLS   Adj. R-squared (uncentered):              0.585
Method:                 Least Squares   F-statistic:                              47.92
Date:                Thu, 07 Mar 2024   Prob (F-statistic):                    4.38e-19
Time:                        16:15:26   Log-Likelihood:                         -18.104
No. Observations:                 100   AIC:                                      42.21
Df Residuals:                      97   BIC:                                      50.02
Df Model:                           3                                                  
Covariance Type:            nonrobust                                                  
                 coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------